In [56]:
import sqlite3
import pprint
import pandas as pd

In [59]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [60]:
cur = conn.cursor()

In [68]:
#explore the database
#get the table names in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
Tables = cur.fetchall()                    

def col_name(table):
    cur.execute("SELECT * FROM {}".format(table))
    col_name_list = [tuple[0] for tuple in cur.description]
    pprint.pprint(col_name_list)

# first table
first_table = Tables[0][0]                    
print(first_table)
# first table column names
col_name(first_table)

# second table 
second_table = Tables[1][0]                    
print(second_table)
# second table column names
col_name(second_table)

# third table 
third_table = Tables[2][0]                    
print(third_table)
# third table column names
col_name(third_table)

Bookings
['bookid', 'facid', 'memid', 'starttime', 'slots']
Facilities
['facid',
 'name',
 'membercost',
 'guestcost',
 'initialoutlay',
 'monthlymaintenance']
Members
['memid',
 'surname',
 'firstname',
 'address',
 'zipcode',
 'telephone',
 'recommendedby',
 'joindate']


In [64]:
def query(sqltext):
    cur.execute(sqltext)
    rows=cur.fetchall()
    columns_tuple=cur.description
    columns_list = [field_tuple[0] for field_tuple in columns_tuple]
    df=pd.DataFrame(rows,
                    columns = columns_list)
    return df

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [65]:
sql_1="SELECT * FROM (SELECT name, SUM(cost) AS total_revenue FROM (SELECT f.name, CASE b.memid WHEN 0 THEN f.guestcost*b.slots ELSE f.membercost*b.slots END AS cost FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid=f.facid) AS subqurey GROUP BY name) AS S2 WHERE total_revenue<1000 ORDER BY total_revenue"
query(sql_1)

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [66]:
sql_2='SELECT m1.firstname AS member_firstname, m1.surname AS member_surname, m2.firstname AS recommendedby_firstname, m2.surname AS recommendedby_surname FROM Members m1 INNER JOIN Members m2 ON m1.recommendedby=m2.memid WHERE m1.recommendedby IS NOT NULL ORDER BY member_surname, member_firstname'
query(sql_2)

,member_firstname,member_surname,recommendedby_firstname,recommendedby_surname
0,Florence,Bader,Ponder,Stibbons
1,Anne,Baker,Ponder,Stibbons
2,Timothy,Baker,Jemima,Farrell
3,Tim,Boothe,Tim,Rownam
4,Gerald,Butters,Darren,Smith
5,Joan,Coplin,Timothy,Baker
6,Erica,Crumpet,Tracy,Smith
7,Nancy,Dare,Janice,Joplette
8,Matthew,Genting,Gerald,Butters
9,John,Hunt,Millicent,Purview


Q12: Find the facilities with their usage by member, but not guests

In [69]:
sql_3='SELECT f.name ,SUM(b.slots) AS member_usage FROM Bookings b LEFT JOIN Facilities f ON b.facid=f.facid WHERE b.memid!=0 GROUP BY name'
query(sql_3)

,name,member_usage
0,Badminton Court,1086
1,Massage Room 1,884
2,Massage Room 2,54
3,Pool Table,856
4,Snooker Table,860
5,Squash Court,418
6,Table Tennis,794
7,Tennis Court 1,957
8,Tennis Court 2,882


Q13: Find the facilities usage by month, but not guests

In [73]:
sql_4="SELECT strftime('%Y-%m',starttime) AS months, f.name ,SUM(b.slots) AS member_usage FROM Bookings b LEFT JOIN Facilities f ON b.facid=f.facid WHERE b.memid<>0 GROUP BY name, months"
query(sql_4)

,months,name,member_usage
0,2012-07,Badminton Court,165
1,2012-08,Badminton Court,414
2,2012-09,Badminton Court,507
3,2012-07,Massage Room 1,166
4,2012-08,Massage Room 1,316
5,2012-09,Massage Room 1,402
6,2012-07,Massage Room 2,8
7,2012-08,Massage Room 2,18
8,2012-09,Massage Room 2,28
9,2012-07,Pool Table,110
